In [2]:
from bs4 import BeautifulSoup
import requests
import string
import random
import re
import pandas as pd
from openpyxl import load_workbook,Workbook
from IPython.display import clear_output
clear_output(wait=True)

In [3]:
# utwórz za pomocą openpyxl trzy arkusze o nazwach “Giełda”, “Linki” i “Filmweb”
wb = Workbook()
name = r'lab4.xlsx'
wb.save(name)
def save_to_excel(data,sheet):
    writer = pd.ExcelWriter(name,engine="openpyxl")
    book = load_workbook(filename=name)
    writer.book=book
    data.to_excel(writer, sheet_name=sheet)
    writer.save()



In [4]:
'''pod adresem znajdziesz notowania giełdowe spółki CD Projekt Red,
wygeneruj 5 losowych kodów 3-literowych,
generowane są trzyliterowe kody do momentu aż trafi się 5, które istnieją.
inna opcja - na stronie z błędem 404 (dla nieistniejącego kodu) jest link do najbardziej pasującego rekordu, można go wyciągnąć za pomocą BS4 i stworzyć kolejne zapytanie,
'''

codes = []
url = "https://stooq.pl/q/?s="
while len(codes)!=5:
    code = ''.join([random.choice(string.ascii_lowercase) for _ in range(3)])
    request = requests.get(url+code)
    soup=BeautifulSoup(request.text,"html.parser")
    if "nie istnieje" not in soup.get_text():
        codes.append(code)

In [5]:
'''napisz kod, który dla wygenerowanych wyżej trzyliterowych kodów spółki wyświetli ich aktualną cenę (Kurs), procentową zmianę (Zmiana) oraz liczbę transakcji (Transakcje),
'''
data = {"Kody":codes,
                     "Kursy":[],
                     "Zmiana":[],
                     "Transakcje":[]}
url = "https://stooq.pl/q/?s="
for code in codes:
    request = requests.get(url+code)
    soup = BeautifulSoup(request.text,"html.parser")
    print(soup.find(text="Kurs"))
    course = float(soup.find(text="Kurs").parent.find("span").get_text())
    data["Kursy"].append(course)
    changeText = soup.find(text="Zmiana").parent.find(id="aq_"+code+"_m3").get_text()
    change = float(re.sub("[()%]","",changeText))
    data["Zmiana"].append(change)
    transactionText = soup.find(text = "Transakcje").parent.find("span").get_text().replace(" ","")
    transaction = int(transactionText) if transactionText else 0
    data["Transakcje"].append(transaction)





Kurs
Kurs
Kurs
Kurs
Kurs


In [6]:
#zapisz wyniki do arkusza ‘Giełda’
df=pd.DataFrame(data)
save_to_excel(df,"Giełda")

In [7]:
#dla wybranej strony internetowej napisz kod, który połączy się ze stroną, znajdzie wszystkie linki (co najmniej 10) na stronie (znacznik ‘a’ posiadający atrybut ‘href’), a następnie zapisze je do arkusza ‘Linki’
data = {"Linki":[]}
url = "https://www.python.org"
request = requests.get(url)
soup = BeautifulSoup(request.text,"html.parser")
links = soup.find_all('a',href=True)
for link in links:
    x = str(link.get("href"))
    if x not in data["Linki"]:
        if "http" not in x:
            x = url+x
        data["Linki"].append(x)

df = pd.DataFrame(data)
save_to_excel(df,"Linki")

In [8]:
'''dla ustalonego linku do filmu na Filmwebie, np. tego, napisz kod, który zwróci:
reżysera,
datę premiery,
boxoffice,
ocenę filmu.
zapisz uzyskane wyniki do arkusza ‘Filmweb’'''
film = {}
url = "https://www.filmweb.pl/film/Diabe%C5%82+wcielony-2020-809318"
request = requests.get(url)
soup = BeautifulSoup(request.text,"html.parser")

director = soup.find(itemprop="director").text.strip()
film["Reżyser"] = [director]

premiere_date = soup.find("span", {"class": "block"}).text.strip()
film["Data premiery"] = [premiere_date]

boxoffice = soup.find("div", {"class": "filmRating filmRating--hasPanel"}).attrs.get("data-count")
film["boxoffice"] = [boxoffice]

rating = float(soup.find("div", {"class": "filmRating filmRating--hasPanel"}).attrs.get("data-rate"))
film["rating"] = [round(rating,1)]


df = pd.DataFrame(film)

save_to_excel(df,"Filmweb")